In [75]:
import os 
import json
from dotenv import main

from langchain_groq import ChatGroq
from dotenv import main
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate  
from langchain.chains import LLMChain


In [76]:
_ = main.load_dotenv(main.find_dotenv())
api_key, llama_model = os.getenv("GROQ_API_KEY"), os.getenv("GROQ_LLama")

In [77]:
project_title = "Predictive Maintenance Using Machine Learning for Industrial Equipment" 
project_description = """
Project Description:

In industrial settings, machinery breakdowns can cause significant downtime, resulting in reduced productivity and increased operational costs. Traditional maintenance methods are either reactive, where repairs are made after a failure occurs, or preventive, where regular checks are performed without necessarily detecting a failure beforehand. Both approaches are inefficient and can lead to unnecessary downtime or excessive maintenance costs.

This project aims to develop a Predictive Maintenance System using machine learning models that can anticipate equipment failures before they happen. By leveraging historical sensor data, operational logs, and maintenance records, the system will predict when a machine is likely to fail, enabling maintenance teams to address potential issues in advance. This proactive approach minimizes downtime, reduces maintenance costs, and extends the lifespan of equipment.

Objectives:
1. Data Collection & Preprocessing:
   - Gather historical data from industrial sensors (temperature, vibration, pressure, etc.), machine logs, and maintenance records.
   - Clean, normalize, and preprocess the data to remove noise, handle missing values, and scale features.
   
2. Feature Engineering:
   - Extract relevant features from raw sensor data, such as peak temperatures, vibration thresholds, and pressure changes, that may indicate potential machine failure.
   - Generate time-series features to capture trends and patterns in equipment performance.

3. Model Development:
   - Implement supervised learning algorithms (e.g., Random Forest, Gradient Boosting, Support Vector Machine) to classify and predict equipment failure.
   - Explore unsupervised learning methods (e.g., K-means clustering, Autoencoders) to detect anomalous behavior in equipment that could signal an impending breakdown.
   - Use time-series models (e.g., LSTM, ARIMA) to forecast the time-to-failure for each machine.

4. Model Evaluation & Optimization:
   - Evaluate the models using accuracy, precision, recall, and F1 score to ensure that they correctly predict failures while minimizing false positives.
   - Perform hyperparameter tuning and cross-validation to optimize model performance.

5. Deployment:
   - Build a user-friendly dashboard to visualize the real-time status of equipment, predictions of potential failures, and recommended maintenance actions.
   - Integrate the machine learning model with IoT sensors for real-time monitoring and predictive analytics.
   - Enable alerts and notifications for the maintenance team when the system predicts a failure is likely to occur.

6. Business Impact Analysis:
   - Estimate potential cost savings from reducing unplanned downtime and unnecessary maintenance.
   - Measure improvements in equipment utilization and productivity as a result of implementing predictive maintenance.

Expected Outcomes:
- A machine learning model that can accurately predict equipment failures based on sensor data and operational logs.
- A real-time monitoring system that integrates with existing industrial infrastructure, providing actionable insights for proactive maintenance.
- A reduction in downtime and maintenance costs, leading to more efficient industrial operations.

This project demonstrates how machine learning can be applied to enhance industrial maintenance processes, leading to significant operational improvements and cost savings.
"""

In [78]:
from pydantic import BaseModel, Field
from typing import List

class ChecklistItem(BaseModel):
    task: str = Field(..., description="A description of the task in the checklist.")
    time_in_days: int = Field(..., description="Time required to complete the task in days.")

class Milestone(BaseModel):
    milestone_title: str = Field(..., description="The title or name of the milestone.")
    checklist: List[ChecklistItem] = Field(..., description="A list of tasks with time estimates under this milestone.")

class ProjectMilestones(BaseModel):
    project_title: str = Field(..., description="Title of the project.")
    description: str = Field(..., description="A short description of the project.")
    milestones: List[Milestone] = Field(..., description="A list of milestones with checklists and time estimates.")


In [79]:
llm = ChatGroq(temperature=0, model=llama_model, api_key=api_key)

parser = JsonOutputParser(pydantic_object=ProjectMilestones)
input_prompt = f"""
Generate a structured milestone plan for a project:

- Project Title: {project_title}
- Description: {project_description}

Please provide the output in the following format:
{parser.get_format_instructions()}
"""


In [80]:
structured_llm = llm.with_structured_output(ProjectMilestones, method="json_mode")
output = structured_llm.invoke(input_prompt)

try:
    json_output = json.dumps(output)
    validated_output = ProjectMilestones.model_validate_json(json_output)
    with open('../milestone/milestone.json', 'w') as file:
        json.dump(output, file, indent=4)
except UnicodeEncodeError as e:
    print("Output does not match the expected structure:", e)
